<a href="https://colab.research.google.com/github/harinadh12/Freetime_NLP/blob/main/BERT_MLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers

In [2]:
from transformers import BertTokenizer, BertForMaskedLM
import torch
import pandas as pd

In [78]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForMaskedLM.from_pretrained('bert-base-uncased')

loading file https://huggingface.co/bert-base-uncased/resolve/main/vocab.txt from cache at /root/.cache/huggingface/transformers/45c3f7a79a80e1cf0a489e5c62b43f173c15db47864303a55d623bb3c96f72a5.d789d64ebfe299b0e416afc4a169632f903f693095b4629a7ea271d5a0cf2c99
loading file https://huggingface.co/bert-base-uncased/resolve/main/added_tokens.json from cache at None
loading file https://huggingface.co/bert-base-uncased/resolve/main/special_tokens_map.json from cache at None
loading file https://huggingface.co/bert-base-uncased/resolve/main/tokenizer_config.json from cache at /root/.cache/huggingface/transformers/c1d7f0a763fb63861cc08553866f1fc3e5a6f4f07621be277452d26d71303b7e.20430bd8e10ef77a7d2977accefe796051e01bc2fc4aa146bc862997a1a15e79
loading configuration file https://huggingface.co/bert-base-uncased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/3c61d016573b14f7f008c02c4e51a366c67ab274726fe2910691e2a761acf43e.37395cee442ab11005bcd270f3c34464dc1704b715b5d7

In [82]:
data = pd.read_csv('unredactor.tsv', on_bad_lines='skip',sep='\t', header=None)
data.columns =['author','type','label','text']

In [83]:
data.text[1]

"I'll admit that I was reluctant to see it because from what I knew of ██████████████ he was only able to do comedy."

In [84]:
data.shape

(1939, 4)

In [85]:
train_data = data[data.type=='training']

In [86]:
train_data = train_data.apply(lambda x : x['text'].replace('█'*len(x['label']),x['label']), axis=1).tolist()

In [87]:
inputs = tokenizer(train_data,return_tensors='pt', truncation=True,max_length=50,padding=True)
inputs.keys()
inputs

{'input_ids': tensor([[ 101, 1045, 2481,  ...,    0,    0,    0],
        [ 101, 1045, 1005,  ...,    0,    0,    0],
        [ 101, 1996, 3772,  ...,    0,    0,    0],
        ...,
        [ 101, 2023, 2003,  ..., 1045, 1036,  102],
        [ 101, 2202, 9901,  ...,    0,    0,    0],
        [ 101, 1045, 3446,  ...,  102,    0,    0]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 1, 0, 0]])}

In [88]:
inputs['labels'] = inputs.input_ids.detach().clone()

In [89]:
inputs.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'labels'])

In [90]:
# create random array of floats in equal dimension to input_ids
rand = torch.rand(inputs.input_ids.shape)


In [91]:
mask_arr = (rand < 0.15) * (inputs.input_ids != 101) * (inputs.input_ids != 102)
mask_arr

tensor([[False, False, False,  ...,  True, False, False],
        [False, False, False,  ..., False, False, False],
        [False, False, False,  ..., False, False, False],
        ...,
        [False, False, False,  ...,  True, False, False],
        [False, False, False,  ..., False, False, False],
        [False, False, False,  ..., False, False, False]])

In [92]:
inputs.input_ids[2].shape

torch.Size([50])

In [93]:
selection = []

for i in range(inputs.input_ids.shape[0]):
    selection.append(
        torch.flatten(mask_arr[i].nonzero()).tolist()
    )

In [94]:
for i in range(inputs.input_ids.shape[0]):
    inputs.input_ids[i, selection[i]] = 103

In [95]:
inputs.input_ids

tensor([[ 101, 1045, 2481,  ...,  103,    0,    0],
        [ 101, 1045, 1005,  ...,    0,    0,    0],
        [ 101, 1996, 3772,  ...,    0,    0,    0],
        ...,
        [ 101, 2023, 2003,  ...,  103, 1036,  102],
        [ 101, 2202, 9901,  ...,    0,    0,    0],
        [ 101, 1045, 3446,  ...,  102,    0,    0]])

In [96]:
class MeditationsDataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings
    def __getitem__(self, idx):
        return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
    def __len__(self):
        return len(self.encodings.input_ids)

In [97]:
dataset = MeditationsDataset(inputs)

In [98]:
loader = torch.utils.data.DataLoader(dataset, batch_size=16, shuffle=True)

In [99]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
# and move our model over to the selected device
model.to(device)
# activate training mode
model.train()

BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=Tr

In [100]:
from transformers import AdamW
# initialize optimizer
optim = AdamW(model.parameters(), lr=5e-5)

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


Trainer API

In [102]:
from transformers import TrainingArguments

args = TrainingArguments(
    output_dir='out',
    per_device_train_batch_size=16,
    num_train_epochs=5,

)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [103]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=dataset
)

In [104]:
trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 1078
  Num Epochs = 5
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 340
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  """


Step,Training Loss




Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=340, training_loss=0.3427129408892463, metrics={'train_runtime': 3125.2029, 'train_samples_per_second': 1.725, 'train_steps_per_second': 0.109, 'total_flos': 138542377050000.0, 'train_loss': 0.3427129408892463, 'epoch': 5.0})

In [43]:
# tokenizer = AutoTokenizer.from_pretrained(model_name)
# model = AutoModelForMaskedLM.from_pretrained(model_name)
# unmasker = pipeline('fill-mask', model=model, tokenizer=tokenizer)

In [44]:
model.save_pretrained('./')

Configuration saved in ./config.json
Model weights saved in ./pytorch_model.bin


In [45]:
trainer.save_model('./')

Saving model checkpoint to ./
Configuration saved in ./config.json
Model weights saved in ./pytorch_model.bin


In [ ]:
model.eval()

In [66]:
test_token = tokenizer("I'll admit that I was reluctant to see it because from what I knew of [MASK] he was only able to do comedy.",return_tensors='pt', truncation=True,max_length=50,padding=True)

In [105]:
trainer.save_model("./out/model2/")

Saving model checkpoint to ./out/model2/
Configuration saved in ./out/model2/config.json
Model weights saved in ./out/model2/pytorch_model.bin


In [113]:
bert_model = BertForMaskedLM.from_pretrained("./out/model2")

loading configuration file ./out/model2/config.json
Model config BertConfig {
  "_name_or_path": "bert-base-uncased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "torch_dtype": "float32",
  "transformers_version": "4.18.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading weights file ./out/model2/pytorch_model.bin
All model checkpoint weights were used when initializing BertForMaskedLM.

All the weights of BertForMaskedLM were initialized from the model checkpoint at ./out/model2.
If your task is similar to the task t

In [114]:
text = "I'll admit that I was reluctant to see it because from what I knew of [MASK] he was only able to do comedy."

In [115]:
from torch.nn import functional as F
input = tokenizer.encode_plus(text, return_tensors = "pt")
mask_index = torch.where(input["input_ids"][0] == tokenizer.mask_token_id)
output = bert_model(**input)
logits = output.logits
softmax = F.softmax(logits, dim = -1)
mask_word = softmax[0, mask_index, :]
top_10 = torch.topk(mask_word, 10, dim = 1)[1][0]
for token in top_10:
   word = tokenizer.decode([token])
   new_sentence = text.replace(tokenizer.mask_token, word)
   print(new_sentence)

I'll admit that I was reluctant to see it because from what I knew of him he was only able to do comedy.
I'll admit that I was reluctant to see it because from what I knew of eddie he was only able to do comedy.
I'll admit that I was reluctant to see it because from what I knew of flynn he was only able to do comedy.
I'll admit that I was reluctant to see it because from what I knew of dylan he was only able to do comedy.
I'll admit that I was reluctant to see it because from what I knew of drew he was only able to do comedy.
I'll admit that I was reluctant to see it because from what I knew of kevin he was only able to do comedy.
I'll admit that I was reluctant to see it because from what I knew of it he was only able to do comedy.
I'll admit that I was reluctant to see it because from what I knew of jack he was only able to do comedy.
I'll admit that I was reluctant to see it because from what I knew of paul he was only able to do comedy.
I'll admit that I was reluctant to see it bec